In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl
import flopy

In [ ]:
# import containerized functionality from defaults.py
from defaults import *

### Parallel settings

1. Set voronoi to `True` to evaluate performance of Voronoi grid base and parallel models. metis cannot be `True` if voronoi is `True`.
2. Set metis to `True` to evaluate performance of structured grid base and parallel models split with Metis. voronoi cannot be `True` if metis is `True`.

In [ ]:
voronoi = False
metis = True

In [ ]:
paths = get_available_workspaces(metis=metis, voronoi=voronoi)
paths

In [ ]:
processors = get_simulation_processors(metis=metis, voronoi=voronoi)
processors

In [ ]:
mean_runtimes = []
mean_formulatetimes = []
mean_solutiontimes = []
mean_iterations = []
for path in paths:
    list_files = get_simulation_listfiles(path)
    mean_runtimes.append(np.mean([SimulationData(list_file).get_model_runtime() for list_file in list_files]))
    mean_formulatetimes.append(np.mean([SimulationData(list_file).get_formulate_time() for list_file in list_files]))
    mean_solutiontimes.append(np.mean([SimulationData(list_file).get_solution_time() for list_file in list_files]))
    mean_iterations.append(np.mean([SimulationData(list_file).get_total_iterations() for list_file in list_files]))
    

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(figwidth, figheight*0.9), constrained_layout=True, sharex=True)
axes = axes.flatten()

ax = axes[0]
ax.plot(processors, mean_runtimes, lw=0.5, color="black", marker="o", ms=4, mfc="none", mec="black")
ax.set_ylabel("Runtime (sec)")

ax = axes[1]
ax.plot(processors, mean_iterations, lw=0.5, color="black", marker="o", ms=4, mfc="none", mec="black")
ax.set_ylabel("Number of iterations")

ax = axes[2]
ax.plot(processors, mean_formulatetimes, lw=0.5, color="black", marker="o", ms=4, mfc="none", mec="black")
ax.set_xlabel("Number of CPUs")
ax.set_ylabel("Formulation time (sec)")

ax = axes[3]
ax.plot(processors, mean_solutiontimes, lw=0.5, color="black", marker="o", ms=4, mfc="none", mec="black")
ax.set_xlabel("Number of CPUs")
ax.set_ylabel("Solution time (sec)");